# Dataloading (1B+ Training Pairs)

## This notebook collects and uploads all 50 relevant sentence embedding datasets to S3 as .json.gz files where each line contains one training record

In [62]:
import os
import pandas as pd

os.chdir('/home/ec2-user/SageMaker')

In [63]:
# install git-lfs
os.system('sudo amazon-linux-extras install epel -y')
os.system('sudo yum-config-manager --enable epel')
os.system('sudo yum install git-lfs -y')
os.system('git lfs install')

Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
Cleaning repos: amzn2-core amzn2extra-docker amzn2extra-epel
              : amzn2extra-kernel-5.10 amzn2extra-python3.8 centos-extras
              : copr:copr.fedorainfracloud.org:vbatts:shadow-utils-newxidmap
              : docker-ce-stable libnvidia-container neuron
21 metadata files removed
15 sqlite files removed
0 metadata files removed
Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock


https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.


62 packages excluded due to repository priority protections
Resolving Dependencies
--> Running transaction check
---> Package epel-release.noarch 0:7-11 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package              Arch           Version      Repository               Size
Installing:
 epel-release         noarch         7-11         amzn2extra-epel          15 k

Transaction Summary
Install  1 Package

Total download size: 15 k
Installed size: 24 k
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction


  Installing : epel-release-7-11.noarch                                     1/1 
  Verifying  : epel-release-7-11.noarch                                     1/1 

Installed:
  epel-release.noarch 0:7-11                                                    

Complete!
Installing epel-release
  0  ansible2                 available    \
        [ =2.4.2  =2.4.6  =2.8  =stable ]
  2  httpd_modules            available    [ =1.0  =stable ]
  3  memcached1.5             available    \
        [ =1.5.1  =1.5.16  =1.5.17 ]
  6  postgresql10             available    [ =10  =stable ]
  9  R3.4                     available    [ =3.4.3  =stable ]
 10  rust1                    available    \
        [ =1.22.1  =1.26.0  =1.26.1  =1.27.2  =1.31.0  =1.38.0
          =stable ]
 18  libreoffice              available    \
        [ =5.0.6.2_15  =5.3.6.1  =stable ]
 19  gimp                     available    [ =2.8.22 ]
 20  docker=latest            enabled      \
        [ =17.12.1  =18.03.1  =18.06.1  =

https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.
http://mirror.es.its.nyu.edu/epel/7/x86_64/repodata/repomd.xml: [Errno 12] Timeout on http://mirror.es.its.nyu.edu/epel/7/x86_64/repodata/repomd.xml: (28, 'Failed to connect to mirror.es.its.nyu.edu port 80 after 5001 ms: Timeout was reached')
Trying other mirror.


286 packages excluded due to repository priority protections
Resolving Dependencies
--> Running transaction check
---> Package git-lfs.x86_64 0:2.10.0-2.el7 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package           Arch             Version                Repository      Size
Installing:
 git-lfs           x86_64           2.10.0-2.el7           epel           3.7 M

Transaction Summary
Install  1 Package

Total download size: 3.7 M
Installed size: 13 M


Importing GPG key 0x352C64E5:
 Userid     : "Fedora EPEL (7) <epel@fedoraproject.org>"
 Fingerprint: 91e9 7d7c 4a5e 96f1 7f3e 888f 6a2f aea2 352c 64e5
 Package    : epel-release-7-11.noarch (@amzn2extra-epel)
 From       : /etc/pki/rpm-gpg/RPM-GPG-KEY-EPEL-7


Public key for git-lfs-2.10.0-2.el7.x86_64.rpm is not installed
Retrieving key from file:///etc/pki/rpm-gpg/RPM-GPG-KEY-EPEL-7
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : git-lfs-2.10.0-2.el7.x86_64                                  1/1 
  Verifying  : git-lfs-2.10.0-2.el7.x86_64                                  1/1 

Installed:
  git-lfs.x86_64 0:2.10.0-2.el7                                                 

Complete!
Git LFS initialized.


0

In [3]:
# clone relevant datasets' github repositories
stacks = ['stackexchange_title_body_jsonl', #25.3M 
          'stackexchange_titlebody_best_voted_answer_jsonl', #4.75M
          'stackexchange_title_best_voted_answer_jsonl', #4.75M
          'stackexchange_titlebody_best_and_down_voted_answer_jsonl'] #210K

os.environ['GIT_LFS_SKIP_SMUDGE'] = "1"

# clone stackexchange repos
for stack in stacks:
    os.system(f'git clone https://huggingface.co/datasets/flax-sentence-embeddings/{stack}')
# clone reddit repo
os.system('git clone https://huggingface.co/datasets/sentence-transformers/reddit-title-body')
# clone 1B+ sentence embeddings repo (this one is just for reference)
os.system('git clone https://github.com/AntoineSimoulin/1B_sentence_embeddings')

Cloning into 'stackexchange_title_body_jsonl'...
Cloning into 'stackexchange_titlebody_best_voted_answer_jsonl'...
Cloning into 'stackexchange_title_best_voted_answer_jsonl'...
Cloning into 'stackexchange_titlebody_best_and_down_voted_answer_jsonl'...
Cloning into 'reddit-title-body'...
Cloning into '1B_sentence_embeddings'...


0

In [ ]:
# DOWNLOAD GITHUB DATASETS (STACKEXCHANGE (https://huggingface.co/flax-sentence-embeddings) & REDDIT (https://huggingface.co/datasets/sentence-transformers/reddit-title-body))

# these are the files marked as unsafe by HuggingFace when viewing the each of the datasets' pages
unsafe = [["serverfault.com.jsonl.gz", "security.stackexchange.com.jsonl.gz"],
          ["monero.stackexchange.com.jsonl.gz", "serverfault.com.jsonl.gz", "security.stackexchange.com.jsonl.gz"],
          ["elementaryos.stackexchange.com.jsonl.gz", "monero.stackexchange.com.jsonl.gz", "security.stackexchange.com.jsonl.gz"],
          [""]]

file_counts = []
print('Downloading {:,} StackExchange GitHub datasets into s3://lodestone-rnd/data/'.format(len(stacks)))
for i, stack in enumerate(stacks):
    # get the names of all the files in the repository that are not unsafe
    files = [file for file in os.listdir(f'/home/ec2-user/SageMaker/{stack}') if file.endswith(".jsonl.gz")==True if file not in unsafe[i]]
    file_counts.append(len(files))
    os.chdir(f'/home/ec2-user/SageMaker/{stack}')
    print('Downloading dataset {} ({} files) ... '.format(stack, len(files)), end='', flush=True)
    # sequentially pull each dataset from git lfs, stream it to S3, and then delete the local copy to free up disk memory
    for file_name in files:
        os.system(f'git lfs pull --include={file_name}')
        os.system(f'aws s3 cp {file_name} s3://lodestone-rnd/data/{stack}/{file_name[:-9] + ".json.gz"}')
        os.remove(file_name)
        os.system('rm -r .git/lfs/objects/*')
        if len(os.listdir('.git/objects/pack')) == 4:
            os.system('ls -t .git/objects/pack/* | head -2 | xargs rm --')
    print('\033[32m' + 'Done' + '\033[0m')
print(f'Total files uploaded: {sum(file_counts)}')

print("\n")

print('Downloading {:,} Reddit GitHub dataset into s3://lodestone-rnd/data/'.format(1))
# get the names of all the files in the repository
files = [file for file in os.listdir(f'/home/ec2-user/SageMaker/reddit-title-body') if file.endswith(".jsonl.gz")==True]
os.chdir(f'/home/ec2-user/SageMaker/reddit-title-body')
print('Downloading dataset {} ({} files) ... '.format("reddit-title-body", len(files)), end='', flush=True)
# sequentially pull each dataset from git lfs, stream it to S3, and then delete the local copy to free up disk memory
for file_name in files:
    os.system(f'git lfs pull --include={file_name}')
    os.system(f'aws s3 cp {file_name} s3://lodestone-rnd/data/reddit-title-body/{file_name[:-9] + ".json.gz"}')
    os.remove(file_name)
    os.system('rm -r .git/lfs/objects/*')
    if len(os.listdir('.git/objects/pack')) == 4:
            os.system('ls -t .git/objects/pack/* | head -2 | xargs rm --')
print('\033[32m' + 'Done' + '\033[0m')
print(f'Total files uploaded: {len(files)}')

os.chdir('/home/ec2-user/SageMaker')

print("\n")

# DOWNLOAD HUGGINGFACE DATASETS (https://huggingface.co/datasets/sentence-transformers/embedding-training-data)

# read dataset information from HuggingFace_datasets.tsv
datasets = pd.read_csv(
    'HuggingFace_datasets.tsv',
    index_col=0,
    sep='\t',
    dtype={
          'Description': str,
          'Size (#Pairs)': str,
          'Performance': float,
          'Download link': str,
          'Source': str})
datasets['Size (#Pairs)'] = datasets['Size (#Pairs)'].str.replace(',', '').astype(int)
datasets = datasets.to_dict(orient='index')

print('Downloading {:,} HuggingFace datasets into s3://lodestone-rnd/data/'.format(len(datasets)))

# stream each of the datasets from the URL provided into S3
# (note that S2ORC_citations_abstracts is larger than 50GB and therefore requires the expected size to be passed into the command line as well)
for d in datasets.keys():
    print('Downloading dataset {} ({:,} pairs) ... '.format(d, datasets[d]['Size (#Pairs)']), end='', flush=True)
    if d == "S2ORC_citations_abstracts":
        os.system(f'wget -qO- {datasets[d]["Download link"]} | aws s3 cp - s3://lodestone-rnd/data/{d + ".json.gz"} --expected-size 120259084288')
    else:
        os.system(f'wget -qO- {datasets[d]["Download link"]} | aws s3 cp - s3://lodestone-rnd/data/{d + ".json.gz"}')
    print('\033[32m' + 'Done' + '\033[0m')

print("\n")

# DOWNLOAD GOOGLE SHEETS DATASETS (https://docs.google.com/spreadsheets/d/1vXJrIg38cEaKjOG5y4I4PQwAQFUmCkohbViJ9zj_Emg/edit#gid=0)

# read dataset information from GoogleSheets_datasets.tsv
datasets = pd.read_csv(
    'GoogleSheets_datasets.tsv',
    index_col=0,
    sep='\t',
    dtype={
          'Description': str,
          'Size (#Pairs)': str,
          'Performance': float,
          'Download link': str,
          'Source': str})
datasets['Size (#Pairs)'] = datasets['Size (#Pairs)'].str.replace(',', '').astype(int)
datasets = datasets.to_dict(orient='index')

print('Downloading {:,} 1B+ Google Sheets datasets into s3://lodestone-rnd/data/'.format(len(datasets)))

# stream each of the datasets from the URL provided into S3
for d in datasets.keys():
    print('Downloading dataset {} ({:,} pairs) ... '.format(d, datasets[d]['Size (#Pairs)']), end='', flush=True)
    os.system(f'wget -qO- {datasets[d]["Download link"]} | aws s3 cp - s3://lodestone-rnd/data/{d + ".json.gz"}')
    print('\033[32m' + 'Done' + '\033[0m')

print("\n")
    
print(f'Successfully downloaded 50 datasets and {621+12+9+36} files into s3://lodestone-rnd/data/')

upload: ./networkengineering.stackexchange.com.jsonl.gz to s3://lodestone-rnd/data/stackexchange_title_body_jsonl/networkengineering.stackexchange.com.json.gz
upload: ./emacs.stackexchange.com.jsonl.gz to s3://lodestone-rnd/data/stackexchange_title_body_jsonl/emacs.stackexchange.com.json.gz
upload: ./christianity.stackexchange.com.jsonl.gz to s3://lodestone-rnd/data/stackexchange_title_body_jsonl/christianity.stackexchange.com.json.gz
upload: ./bitcoin.stackexchange.com.jsonl.gz to s3://lodestone-rnd/data/stackexchange_title_body_jsonl/bitcoin.stackexchange.com.json.gz
upload: ./academia.stackexchange.com.jsonl.gz to s3://lodestone-rnd/data/stackexchange_title_body_jsonl/academia.stackexchange.com.json.gz
upload: ./music.stackexchange.com.jsonl.gz to s3://lodestone-rnd/data/stackexchange_title_body_jsonl/music.stackexchange.com.json.gz
upload: ./biology.stackexchange.com.jsonl.gz to s3://lodestone-rnd/data/stackexchange_title_body_jsonl/biology.stackexchange.com.json.gz
upload: ./histo

In [1]:
# clean up (remove the cloned repositories)
import shutil
shutil.rmtree("/home/ec2-user/SageMaker/stackexchange_title_body_jsonl")
shutil.rmtree("/home/ec2-user/SageMaker/stackexchange_titlebody_best_voted_answer_jsonl")
shutil.rmtree("/home/ec2-user/SageMaker/stackexchange_title_best_voted_answer_jsonl")
shutil.rmtree("/home/ec2-user/SageMaker/stackexchange_titlebody_best_and_down_voted_answer_jsonl")
shutil.rmtree("/home/ec2-user/SageMaker/reddit-title-body")
shutil.rmtree("/home/ec2-user/SageMaker/1B_sentence_embeddings")